# Homework 1

In [1]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from itertools import combinations 
from statsmodels.stats.inter_rater import fleiss_kappa
import numpy as np
import statistics
import random
%config Completer.use_jedi = False

In [2]:
df = pd.read_csv('../../data/Sentiment_Annotation_2021.csv')
df.head

<bound method NDFrame.head of                     id  annotator_1  annotator_2  annotator_3  majority_vote
0     03bSnISJMiM_0001            1            1            1              1
1     03bSnISJMiM_0002            0            0            0              0
2     03bSnISJMiM_0003            0            0           -1              0
3     03bSnISJMiM_0004           -1            0           -1             -1
4     03bSnISJMiM_0005            0            0            0              0
...                ...          ...          ...          ...            ...
2194  zhpQhgha_KU_0031            0            0            0              0
2195  zhpQhgha_KU_0032            0            0            0              0
2196  zhpQhgha_KU_0033            0            0            0              0
2197  zhpQhgha_KU_0034            0            1            0              0
2198  zhpQhgha_KU_0035            0            0            0              0

[2199 rows x 5 columns]>

In [3]:
# Turn to numpy arrays
df_array = df.values
majority_votes = df_array[:,4].astype(int)
ratings = df_array[:,1:4].astype(int)
ids = df_array[:,0]
n_cat = 3
n_sub = len(ids)
n_rat = 3

## Question 1

In [4]:
# fill the fleiss matrix
fleiss_matrix = np.zeros([n_sub,n_cat])
for i in range(n_sub):
    for j in range(n_rat):
        fleiss_matrix[i][ratings[i][j] + 1] += 1

f_kappa = fleiss_kappa(fleiss_matrix)
print(f'Fleiss Kappa: {f_kappa}')

Fleiss Kappa: 0.6503114154533796


Based on our Fleiss Kappa value and Table 1, we have substantial agreement. 

## Question 2

In [5]:
cohen_kappas = []
combs = combinations(range(ratings.shape[1]), 2) 

# run cohens kappa on each set of combinations C
for comb in combs:
    cohen_kappa =cohen_kappa_score(ratings[:,comb[0]],ratings[:,comb[1]])
    print(f'Cohen Kappa for Annotator {comb[0]+1} and Annotator {comb[1]+1}: {cohen_kappa}')

print()

# run cohens kappa on each annotator with the majority vote
for i in range(ratings.shape[1]):
    cohen_kappa =cohen_kappa_score(ratings[:,i],majority_votes)
    print(f'Cohen Kappa for Annotator {i + 1} and the majority vote: {cohen_kappa}')
    cohen_kappas.append(cohen_kappa)

Cohen Kappa for Annotator 1 and Annotator 2: 0.678474847354384
Cohen Kappa for Annotator 1 and Annotator 3: 0.6915786425927507
Cohen Kappa for Annotator 2 and Annotator 3: 0.5808911741284637

Cohen Kappa for Annotator 1 and the majority vote: 0.8945509304299228
Cohen Kappa for Annotator 2 and the majority vote: 0.7841894954762489
Cohen Kappa for Annotator 3 and the majority vote: 0.7979960274129306


The rank of the annotators is 1, 3, then 2.

## Question 3

In [6]:
# Find new weighted majority
w_majority_votes = []
pot = 0
for i in range(n_sub):
    rat1 = ratings[i][0]
    rat2 = ratings[i][1]
    rat3 = ratings[i][2]
    # If there is a tie in the weighted majority, choose randomly between rating 1 and the other rating
    if rat2 == rat3 and rat1 != rat2:
        w_majority_votes.append(random.choice([rat1, rat2]).astype(int))
        pot+=1
    else:
        w_majority_votes.append(statistics.mode([rat1, rat1, rat2, rat3]).astype(int))

In [7]:
print(f'There were {(w_majority_votes != majority_votes).sum()} changes made')

There were 75 changes made


In [8]:
# write to csv
np.savetxt('WeightedMajorityVote.csv',w_majority_votes,fmt='%i')

When there are ties in the weighted vote, I randomly choose between rating 1 and the other possible rating (i.e. rating 2 and rating 3). This is to balance the expertise of rater 1 and the actual majority of the voters.